In [1]:
import os
import re
import sys
import pandas as pd
from tqdm.auto import tqdm

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [9]:
from src.utils import load_dataset

df = load_dataset()

Данные успешно загружены из ../data/df_cleaned.parquet. Количество записей: 19276


In [10]:
from src.classic_keywords.keyphrase_extraction import extract_yake_keyphrases
from src.search_tf_idf.search import TfidfSearch
from src.classic_keywords.preprocessing import preprocess_text

pd.set_option('display.max_colwidth', 300)
tqdm.pandas()

## 1. Извлечение ключевых фраз: YAKE

In [12]:
print("Извлечение ключевых фраз с помощью YAKE...")
df['yake_keywords'] = df['abstract'].progress_apply(extract_yake_keyphrases)
print("Извлечение завершено.")

Извлечение ключевых фраз с помощью YAKE...


  0%|          | 0/19276 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [14]:
print("Примеры извлеченных ключевых фраз:")
display(df[['abstract', 'yake_keywords']].head())

Примеры извлеченных ключевых фраз:


,abstract,yake_keywords
0,"В статье рассматривается сущность и диагностика морально-психологического состояния личного состава. Отмечается, что морально-психологическое состояние исторически устойчивый термин. Возникновение его в начале 1990-х гг. подтвердилось в настоящее время во всех современных армиях, ему придается о...","[состояния личного состава, статье рассматривается сущность, диагностика морально-психологического состояния, морально-психологического состояния личного, личного состава, статье рассматривается, рассматривается сущность]"
1,"Вводятся в научный оборот результаты палеокарпологического анализа керамики поселения черкаскульской культуры бронзового века Ольховка, расположенного в долине Исети. Обнаружение на черепках отпечатков семян пшеницы и ячменя практически первое прямое свидетельство знакомства черкаскульского насе...","[бронзового века Ольховка, научный оборот результаты, оборот результаты палеокарпологического, результаты палеокарпологического анализа, палеокарпологического анализа керамики, анализа керамики поселения, керамики поселения черкаскульской]"
2,"Статья посвящена одонтологической характеристике населения ямной культурно-исторической общности. На основании одонтологических данных проанализирован состав ямного населения Калмыкии, Астраханской области и Украины. Сделан вывод о морфологической неоднородности ямников и ведущей роли в формиров...","[Статья посвящена одонтологической, ямной культурно-исторической общности, посвящена одонтологической характеристике, одонтологической характеристике населения, характеристике населения ямной, населения ямной культурно-исторической, Статья посвящена]"
3,"В статье рассматривается содержание писем и рассказов фронтовиков, оказывавшее разноплановое влияние на восприятие горожанами поволжских губерний действительности в годы Первой мировой войны. Отмечено его содействие развитию у населения чувства патриотизма. Также на конкретных примерах показано ...","[Первой мировой войны, годы Первой мировой, статье рассматривается содержание, восприятие горожанами поволжских, горожанами поволжских губерний, поволжских губерний действительности, оказывавшее разноплановое влияние]"
4,"В статье представлен анализ системы управления Московских приказов и городов в Смутное время начала XVII в. Реконструкция 700 биографий служащих (дьяков и подьячих) позволила выявить основные признаки бюрократии и показать, что кризис верховной власти не приводит к разрушению сложившихся институ...","[системы управления Московских, управления Московских приказов, статье представлен анализ, представлен анализ системы, анализ системы управления, Московских приказов, городов в Смутное]"


In [15]:
output_dir = '../data'
output_path = os.path.join(output_dir, 'df_cleaned.parquet')
os.makedirs(output_dir, exist_ok=True)

In [16]:
df.to_parquet(output_path, index=False, engine='pyarrow')
print(f"DataFrame успешно сохранен по пути: {output_path}")

DataFrame успешно сохранен по пути: ../data/df_cleaned.parquet


YAKE позволяет быстро получить ключевые фразы, в этом главное преимущество классических методов: простая статистическая обработка позволяет обработать большие корпусы текстов в короткие сроки.

Однако имеет ряд системных недостатков
Например:
Алгоритм слепо извлекает последовательности слов, что часто приводит к обрывкам фраз, которые не являются самостоятельными терминами.
*   `морально-психологического состояния личного` (Пример 0) — чего "личного"? Фраза обрывается.
*   `системы управления Московских` (Пример 4) — чего "Московских"? Приказов. Фраза неполная.
*   `керамики поселения черкаскульской` (Пример 1) — чего "черкаскульской"? Культуры.



Для преодоления этих проблем, таких как отсутствие семантического понимания и генерация шаблонных фраз, далее будет применен подход на основе большой языковой модели, способной к обобщению и пониманию контекста. 